In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import glob
import os
from multiprocessing import Process
import sys
sys.path.append('..')
from modules import utils, constants
from stable_baselines import DQN
# from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.deepq.policies import FeedForwardPolicy
from stable_baselines.common.callbacks import CheckpointCallback

In [3]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
SEED

105

In [4]:
# def custom_architecture(input_tensor, num_actions, net_arch):
    
#     with tf.variable_scope('custom_net', reuse=tf.AUTO_REUSE):
#         layer_1 = tf.layers.dense(input_tensor, units=net_arch[0], activation=tf.nn.relu)
#         layer_2 = tf.layers.dense(layer_1, units=net_arch[1], activation=tf.nn.relu)
#         layer_3 = tf.layers.dense(layer_2, units=net_arch[2], activation=tf.nn.relu)
#         layer_4 = tf.layers.dense(layer_3, units=net_arch[3], activation=tf.nn.relu)
#         output_layer = tf.layers.dense(layer_3, units=num_actions, activation=None)

#     return output_layer

In [5]:
# Custom MLP policy of three layers of size 256, 128, 64 and 32 each
class CustomDQNPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomDQNPolicy, self).__init__(*args, **kwargs,
                                           layers=[128, 64, 32],
                                           layer_norm=False,
                                           feature_extraction='mlp')

In [6]:
# policy_kwargs = {'net_arch': [256, 128, 64]}
# policy_kwargs = dict(net_arch=[256, 128, 64])

In [7]:
def stable_dueling_dqn(X_train, y_train, timesteps, save=False, log_path=None, log_prefix='dueling_dqn', filename=None, per=False):
    '''
    Creates and trains a dueling DQN model
    '''
    if per:
        log_prefix = 'dueling_dqn_per'
    training_env = utils.create_env(X_train, y_train)
    model = DQN(CustomDQNPolicy, training_env, verbose=1, seed=constants.SEED, learning_rate=0.0001, 
                buffer_size=1000000, learning_starts=50000, train_freq=4, target_network_update_freq=10000, 
                exploration_final_eps=0.05, n_cpu_tf_sess=1, double_q=False, prioritized_replay=per)
    
    checkpoint_callback = CheckpointCallback(save_freq=constants.CHECKPOINT_FREQ, save_path=log_path, 
                                             name_prefix=log_prefix)
    model.learn(total_timesteps=timesteps, log_interval=100000, callback=checkpoint_callback)
    if save:
        model.save(f'{log_path}/{filename}.pkl')
    training_env.close()
    return model

In [8]:
def stable_dueling_ddqn(X_train, y_train, timesteps, save=False, log_path=None, log_prefix='dueling_ddqn', filename=None, per=False):
    '''
    Creates and trains a dueling double DQN model
    '''
    if per:
        log_prefix = 'dueling_ddqn_per'
    training_env = utils.create_env(X_train, y_train)
    model = DQN(CustomDQNPolicy, training_env, verbose=1, seed=constants.SEED, learning_rate=0.0001, 
                buffer_size=1000000, learning_starts=50000, train_freq=4, target_network_update_freq=10000, 
                exploration_final_eps=0.05, n_cpu_tf_sess=1, prioritized_replay=per)
    
    checkpoint_callback = CheckpointCallback(save_freq=constants.CHECKPOINT_FREQ, save_path=log_path, 
                                             name_prefix=log_prefix)
    model.learn(total_timesteps=timesteps, log_interval=100000, callback=checkpoint_callback)
    if save:
        model.save(f'{log_path}/{filename}.pkl')
    training_env.close()
    return model


In [9]:
def run_dqn_model(model_type, steps):
    dir_name = f'seed_{SEED}_{steps}'
    parent_dir = f'../models/logs/{model_type}/arch_3_layers'
    path = os.path.join(parent_dir, dir_name)
  
    if model_type == 'dueling_dqn_per':
        model = stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn_per', filename=f'dueling_dqn_pre_{steps}', per=True)
    elif model_type == 'dueling_ddqn_per':
        model = stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn_per', filename=f'dueling_ddqn_per_{steps}', per=True)
    else:
        raise ValueError(f'Unknown model type - {model_type}!')
    return model

In [10]:
train_df = pd.read_csv('../data/train_set_basic.csv')
train_df = train_df.fillna(-1)
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


In [11]:
train_df.label.value_counts()

0    26203
1    24197
Name: label, dtype: int64

In [12]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 23), (50400,))

In [13]:
# model_names = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 
#                'dueling_ddqn_per']
model_names = ['dueling_dqn_per', 'dueling_ddqn_per']
procs = []
steps = int(10e7)

In [14]:
for name in model_names:
    proc = Process(target=run_dqn_model, args=(name, steps))
    procs.append(proc)
    proc.start()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -0.7     |
| steps                   | 469657   |
| success rate            | 0.21     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -0.7     |
| steps                   | 469738   |
| success rate            | 0.21     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes    

--------------------------------------
| % time spent exploring  | 23       |
| episodes                | 1300000  |
| mean 100 episode reward | -0.9     |
| steps                   | 8002151  |
| success rate            | 0.15     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 20       |
| episodes                | 1300000  |
| mean 100 episode reward | -1       |
| steps                   | 8384281  |
| success rate            | 0.17     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 19       |
| episodes                | 1400000  |
| mean 100 episode reward | -1       |
| steps                   | 8482749  |
| success rate            | 0.03     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 17       |
| episodes                | 1500000  |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4000000  |
| mean 100 episode reward | -1       |
| steps                   | 13812859 |
| success rate            | 0.01     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4100000  |
| mean 100 episode reward | -1       |
| steps                   | 14013916 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1700000  |
| mean 100 episode reward | -0.2     |
| steps                   | 14633286 |
| success rate            | 0.64     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4200000  |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6700000  |
| mean 100 episode reward | -0.5     |
| steps                   | 19411124 |
| success rate            | 0.25     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6800000  |
| mean 100 episode reward | -0.8     |
| steps                   | 19625740 |
| success rate            | 0.14     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6900000  |
| mean 100 episode reward | -1       |
| steps                   | 19836390 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 7000000  |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9400000  |
| mean 100 episode reward | -1       |
| steps                   | 25254837 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9500000  |
| mean 100 episode reward | -0.6     |
| steps                   | 25466786 |
| success rate            | 0.22     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9600000  |
| mean 100 episode reward | -1       |
| steps                   | 25691327 |
| success rate            | 0.02     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9700000  |
| mean 100 episode reward | -1       |
| steps                  

...

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 14800000 |
| mean 100 episode reward | -0.1     |
| steps                   | 37977860 |
| success rate            | 0.5      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 14900000 |
| mean 100 episode reward | 0.1      |
| steps                   | 38297294 |
| success rate            | 0.55     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 15000000 |
| mean 100 episode reward | 0.2      |
| steps                   | 38658966 |
| success rate            | 0.65     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3300000  |
| mean 100 episode reward | -0       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4000000  |
| mean 100 episode reward | -0.2     |
| steps                   | 50699336 |
| success rate            | 0.64     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 17200000 |
| mean 100 episode reward | 0.5      |
| steps                   | 49663517 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 17300000 |
| mean 100 episode reward | 0.4      |
| steps                   | 50355676 |
| success rate            | 0.84     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4100000  |
| mean 100 episode reward | 0        |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 19200000 |
| mean 100 episode reward | 0.4      |
| steps                   | 63847085 |
| success rate            | 0.8      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 19300000 |
| mean 100 episode reward | 0.4      |
| steps                   | 64522256 |
| success rate            | 0.79     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 5100000  |
| mean 100 episode reward | -0       |
| steps                   | 66681395 |
| success rate            | 0.73     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 19400000 |
| mean 100 episode reward | 0.5      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6000000  |
| mean 100 episode reward | 0.1      |
| steps                   | 79392072 |
| success rate            | 0.74     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 21400000 |
| mean 100 episode reward | 0.4      |
| steps                   | 77448538 |
| success rate            | 0.8      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 21500000 |
| mean 100 episode reward | 0.6      |
| steps                   | 78045414 |
| success rate            | 0.87     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6100000  |
| mean 100 episode reward | -0.2     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 23600000 |
| mean 100 episode reward | 0.6      |
| steps                   | 89395496 |
| success rate            | 0.88     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6900000  |
| mean 100 episode reward | -0.1     |
| steps                   | 92072190 |
| success rate            | 0.67     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 23700000 |
| mean 100 episode reward | 0.6      |
| steps                   | 89951270 |
| success rate            | 0.85     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 23800000 |
| mean 100 episode reward | 0.5      |
| steps                  

In [15]:
for proc in procs:
    proc.join()
print('All jobs completed and terminated successfully')

All jobs completed and terminated successfully
